In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri May 30 10:54:08 2025

@author: xenomorfo
"""

# %%
import pyodbc
import pandas as pd
import numpy as np
from datetime import date


con_1 = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=158.170.66.56,{1433};"
    f"DATABASE=PROC01ESTUDIO;"
    f"UID=proceso;"
    f"PWD=Estudio.2024;")

print("Conexión exitosa")

####listado de tablas
cursor_1 = con_1.cursor()
cursor_1.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.\
                 TABLES WHERE TABLE_TYPE = 'BASE TABLE';")


for t in cursor_1.fetchall():
    print(t)
    
####listado de campos

cursor_1 = con_1.cursor()
columnas=cursor_1.execute("SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.\
                          COLUMNS WHERE TABLE_NAME='CPP_DR2';")

for c in columnas.fetchall():
    print(c)


MAT=pd.read_sql("""
            SELECT 
                m.rut, 
                m.nombres,
                m.ap_paterno,
                m.sexo,
                m.nacionalidad,
                m.cod_plan,
                m.carrera_programa,
                LEFT(periodo_matricula, 4) AS ANHO_MAT,
                CODIGO_CARRERA=cod_carr_prog,
                LEFT(ingreso_plan, 4) AS ANHO_ING,
                CONCAT(m.rut, '-',LEFT(m.ingreso_plan, 4)) AS ID,
                CONCAT(m.rut, '-', LEFT(m.ingreso_plan, 4), '-', m.cod_plan) AS RUT_ANHO_PLAN,
                CONCAT(LEFT(ingreso_plan, 4), '-',m.cod_plan) AS ANHO_PLAN,
                CONCAT(c.SIES, '-', m.rut) AS SIES_RUT,
                CONCAT(LEFT(m.ingreso_plan, 4),'-' ,c.SIES, '-', m.rut) AS ANHO_SIES_RUT,
                m.via_ingreso,
                m.cod_via,
                m.region,
                m.fecha_nac,
                d.GRUPO_DEPENDENCIA,
                d.INGRESO_PERCAPITA_GRUPO_FA,
                d.PUNTAJE_PONDERADO,
                d.MATEMATICA,
                d.COMP_LECT,
                d.PTJE_NEM,
                d.PTJE_RANKING,
                c.SIES,
                o.Tipo_Carrera,
                o.Jornada,
                o.[Cine-F_13_Área],
                o.Duración_Total,
                o.Nivel_Carrera,
                g.GRATUIDAD,
                f.TIPO AS CAE,
                h.TIPO AS FSCU,
                cc.COD_FAC,
                cc.FACULTAD,
                cc.COD_DEPTO,
                cc.Columna2 AS depto,
                mu.NAC,
                COUNT(m.rut) AS Total,        
                d.INGRESO_PERCAPITA_GRUPO_FA,
                d.PUNTAJE_PONDERADO,
                d.MATEMATICA,
                d.COMP_LECT,
                d.PTJE_NEM,
                d.PTJE_RANKING
            FROM MATRICULA_V2_082025_PARA_TODO m
            LEFT JOIN DEMRE_E_2014_2025 d 
            ON CONCAT(m.rut, '-',LEFT(m.ingreso_plan, 4))=d.ID_ANHO
            LEFT JOIN CPP_DR2 c
            ON CONCAT(LEFT(periodo_matricula,4), '-',m.cod_plan)=c.ANHO_PLAN
            LEFT JOIN OA_SIES_2010_2025_USACH o
            ON CONCAT(c.ANHO, '_', c.SIES)=o.llave
            LEFT JOIN tb_gratuidad g
            ON CONCAT(m.rut, '-',LEFT(m.ingreso_plan, 4))=CONCAT(g.Rut,'-',g.AÑO)
            LEFT JOIN cae f
            ON CONCAT(m.rut, '-', LEFT(m.ingreso_plan, 4), '-', m.cod_plan)=CONCAT(f.RUN, '-',f.AÑO_ING,'-', f.CODIGO_PLAN)
            LEFT JOIN fscu h
            ON CONCAT(m.rut, '-', LEFT(m.ingreso_plan, 4), '-', m.cod_plan)=CONCAT(h.RUN, '-',h.AÑO_ING,'-', h.CODIGO_PLAN)
            LEFT JOIN centro_costo cc
            ON cod_carr_prog = [COD CARRERA]
            LEFT JOIN TABLA_MU mu
            ON CONCAT(LEFT(m.ingreso_plan, 4),'-' ,c.SIES, '-', m.rut) = CONCAT(mu.ANHO_MU, '-', mu.COD_SIES,'-', mu.N_DOC)
            GROUP BY
                m.rut,
                m.nombres,
                m.ap_paterno,
                m.sexo,
                m.nacionalidad,
                m.cod_plan,
                m.carrera_programa,
                LEFT(periodo_matricula, 4),
                CONCAT(m.rut, LEFT(m.ingreso_plan, 4)),
                CONCAT(m.rut, '-', LEFT(m.ingreso_plan, 4), '-', m.cod_plan),
                m.cod_carr_prog,
                CONCAT(LEFT(ingreso_plan, 4), '-',m.cod_plan),
                CONCAT(ANHO, '_', SIES),
                CONCAT(c.SIES, '-', m.rut),
                CONCAT(LEFT(m.ingreso_plan, 4),'-',c.SIES, '-', m.rut),
                m.ingreso_plan,
                m.via_ingreso,
                m.region,
                m.fecha_nac,
                d.GRUPO_DEPENDENCIA,
                d.INGRESO_PERCAPITA_GRUPO_FA,
                d.PUNTAJE_PONDERADO,
                d.MATEMATICA,
                d.COMP_LECT,
                d.PTJE_NEM,
                d.PTJE_RANKING,
                c.SIES,
                o.Tipo_Carrera,
                o.Jornada,
                o.[Cine-F_13_Área],
                o.Duración_Total,
                o.Nivel_Carrera,
                g.GRATUIDAD,
                f.TIPO,
                h.TIPO,
                cc.COD_FAC,
                cc.FACULTAD,
                cc.COD_DEPTO,
                cc.Columna2,
                m.cod_via,
                mu.NAC;""", con_1)

###NIVEL GLOBAL
MAT['NIVEL_GLOBAL']=np.where(MAT['CODIGO_CARRERA']=="UNICIT", "UNICIT",
    np.where(MAT['CODIGO_CARRERA']=="MIDA", "MAGISTER",
    np.where(MAT['CODIGO_CARRERA'].str[0:3]=="MAG","MAGISTER", 
    np.where(MAT['CODIGO_CARRERA'].str[0:3]=="DOC","DOCTORADO",
    np.where(MAT['CODIGO_CARRERA'].str[0:3]=="DIP","DIPLOMADO",
    np.where(MAT['CODIGO_CARRERA'].str[0:3]=="POS","POSTITUTLO","PREGRADO"))))))


MAT['fecha_nac']=pd.to_datetime(MAT['fecha_nac'])

MAT['fecha_nac']=pd.to_datetime(
    np.where(MAT['fecha_nac'] > pd.to_datetime('2025-01-01'), 
         MAT['fecha_nac'] - pd.DateOffset(years=100), 
    np.where(MAT['fecha_nac'] < pd.to_datetime('1910-01-01'), 
         MAT['fecha_nac'] + pd.DateOffset(years=100),MAT['fecha_nac'])))

MAT.info()
MAT=MAT.fillna(999)

MAT['ANHO_ING']=MAT['ANHO_ING'].astype(int)
#MAT['ANHO_ING']=MAT['ANHO_ING'].astype(str)
MAT['ANHO_MAT']=MAT['ANHO_MAT'].astype(int)

### edad
MAT['edad']=(((pd.to_datetime(date.today())-MAT['fecha_nac']).dt.days)/365).round(2)

#MAT[(MAT['rut']==19828443)]\
 #   .groupby(['rut', 'nombres', 'ap_paterno','via_ingreso','CODIGO_CARRERA'])

VIA_ING_ANIO=[81,63,26,29,	21,	66,	50,	15,	20,	49,	17,	60,	72,	43,	71,	10,	
              70,53,30,54,	55,	44,	16,	87,	95,	97,	85,	37,	68,	86,	19,	75,	
              64,62,76,27,	14,	77,	11,	74,	65,	12,	84,	73,	18,	83,	61, 99, 
              88,39]


VIA_ING_ANIO_CIDI=[10,11,12,13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	
                   23,24,25,26,	27,	28, 29, 30, 31, 33,	34,	37,	38,	
                   39,40,41,43,	44,	49,	50,	52,	53,	54,	55, 57, 58,
                   59,60,61,62,	63,	64,	65,	66,	68,	70,	71,	72,	73,	
                   74,75,76,77,	78,	80,	79,	81,	82,	83,	84,	85,	86,	
                   87,88,89,91,	92,	95,	97,	99]



###Definición cohorte
MAT['COH']=np.where((np.isin(MAT['cod_via'], VIA_ING_ANIO).astype(int)==1) & 
         (MAT['ANHO_MAT']==MAT['ANHO_ING']),1,0)

MAT['COH_CIDI']=np.where((np.isin(MAT['cod_via'], VIA_ING_ANIO_CIDI).astype(int)==1) & 
         (MAT['ANHO_MAT']==MAT['ANHO_ING']),1,0)


MAT['id-anonimo']=MAT['rut']*25+MAT['ANHO_MAT']

####Definicion primer anio amplio
MAT['primer_anio']=np.where((MAT['ANHO_MAT']==MAT['ANHO_ING']),1,0)

###Informado SIES

informado_sies_2=pd.read_sql("""SELECT
            SIES_RUT,
            ANHO_SIES_RUT
            FROM TABLA_MU
            WHERE VIG=1 AND UNICIT='NO UNICIT'""",con_1)
            
MAT['INFORMADO_SIES']=MAT['ANHO_SIES_RUT'].isin(informado_sies_2['ANHO_SIES_RUT']).astype(int)


#rut_buscado=int(input("Ingresa rut:"))


###busqueda rut
#if rut_buscado in MAT['rut'].values:
 #   print(MAT.loc[MAT['rut']==rut_buscado, 
  #                            ['rut', 
   #                            'ANHO_ING',
    #                           'ANHO_MAT',
     #                          'fecha_nac',
      #                         'CODIGO_CARRERA',
       #                        'cod_plan',
        #                       'SIES']])
#else: print("no encontrado")


def buscar_rut(MAT):
    rut_buscado = int(input("Ingresa rut: "))
    if rut_buscado in MAT['rut'].values:
        print(MAT.loc[MAT['rut'] == rut_buscado, [
            'rut', 
            'ANHO_ING', 
            'ANHO_MAT', 
            'fecha_nac', 
            'CODIGO_CARRERA', 
            'cod_plan', 
            'PTJE_NEM',
            'GRUPO_DEPENDENCIA',
            'via_ingreso',
            'COH',
            'COH_CIDI'
        ]])
    else:
        print("no encontrado")

# Luego llama:
buscar_rut(MAT)


19828443

Conexión exitosa
('MATRICULA',)
('TITULADOS_V_10',)
('CPP_DR2',)
('CIDI_MAESTRO_PROGRAMAS_WEB',)
('tb_gratuidad',)
('CPP_DR',)
('OA_SIES_2010_2025_USACH',)
('cae',)
('fscu',)
('TB_BECA_DESCRIPCION',)
('cidi_pac',)
('cidi_biblioteca_panel2',)
('cidi_rut_pace',)
('TIT_USACH',)
('tab_paises',)
('PAC_2008_2024_V4',)
('DEMRE_E_2014_2025',)
('TABLA_MU',)
('TITULADOS_rev',)
('VAC_SOBRE_CUPOS',)
('COHORTE_AL_20240502',)
('TITULADOS',)
('RA_CONSOLIDADA',)
('TABLA_MU_CALC',)
('cpp_cc',)
('centro_costo',)
('ADMINISTRATIVOS_2021_2024',)
('PAC_2008_2025',)
('BECA_COMPLETA',)
('OA_SIES_2010_2024_USACH',)
('VRYFIL',)
('MATRICULA_V2_082025_PARA_TODO',)
('INSCRIPCION_2025',)
('MU_PRE_POST_2012_2025',)
('ID', 'varchar')
('COD_PLAN', 'varchar')
('ANHO', 'varchar')
('ANHO_SIES', 'varchar')
('PLAN_NOMBRE', 'varchar')
('JORNADA_USACH', 'varchar')
('JORNADA_SIES', 'varchar')
('ESTADO_JORNADA', 'varchar')
('LLAVE', 'varchar')
('SIES', 'varchar')
('LLAVE_SIES', 'varchar')
('PROGRAMA_SIES', 'varchar')
('OBSERVA

/tmp/ipykernel_2911934/3090514048.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MAT=pd.read_sql("""
/tmp/ipykernel_2911934/3090514048.py:162: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  MAT['fecha_nac']=pd.to_datetime(MAT['fecha_nac'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522180 entries, 0 to 522179
Data columns (total 48 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   rut                         522180 non-null  int64         
 1   nombres                     522180 non-null  object        
 2   ap_paterno                  522180 non-null  object        
 3   sexo                        522180 non-null  object        
 4   nacionalidad                522180 non-null  object        
 5   cod_plan                    522180 non-null  int64         
 6   carrera_programa            522180 non-null  object        
 7   ANHO_MAT                    522180 non-null  object        
 8   CODIGO_CARRERA              522180 non-null  object        
 9   ANHO_ING                    522180 non-null  object        
 10  ID                          522180 non-null  object        
 11  RUT_ANHO_PLAN               522180 non-

/tmp/ipykernel_2911934/3090514048.py:213: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informado_sies_2=pd.read_sql("""SELECT


             rut  ANHO_ING  ANHO_MAT  fecha_nac CODIGO_CARRERA  cod_plan  \
230949  19828443      2017      2017 1997-10-21         INGEST      4702   
300993  19828443      2017      2021 1997-10-21         INGEST      4702   
300994  19828443      2017      2022 1997-10-21         INGEST      4702   
309963  19828443      2017      2018 1997-10-21         INGEST      4702   
334706  19828443      2017      2020 1997-10-21         INGEST      4702   
351042  19828443      2017      2019 1997-10-21         INGEST      4702   

        PTJE_NEM  PTJE_NEM  GRUPO_DEPENDENCIA  via_ingreso  COH  COH_CIDI  
230949     564.0     564.0                2.0  CUPO P.S.U.    1         1  
300993     564.0     564.0                2.0  CUPO P.S.U.    0         0  
300994     564.0     564.0                2.0  CUPO P.S.U.    0         0  
309963     564.0     564.0                2.0  CUPO P.S.U.    0         0  
334706     564.0     564.0                2.0  CUPO P.S.U.    0         0  
351042     

19828443

In [3]:

(
MAT[#(MAT['ANHO_ING']==2024) &
    (MAT['NIVEL_GLOBAL']!="PREGRADO")]
.assign(id = MAT['ANHO_ING']
        .astype(str) + '-' + MAT['cod_plan']
        .astype(str))
.groupby(['NIVEL_GLOBAL',
          #'ANHO_PLAN',
          'cod_plan',
          'FACULTAD',
          'COD_DEPTO',
          'CODIGO_CARRERA',
          'ANHO_MAT', 
          'SIES'])['rut']
.nunique()
.reset_index(name= "total")
#.to_clipboard()
)

(
MAT.groupby(['CODIGO_CARRERA', 'SIES'])
.size()
.reset_index(name="tot")
.to_clipboard()
)
#
# (
#MAT.groupby(['cod_plan', 'SIES', 'ANHO_MAT'])
#.size()
#.reset_index(name='counts')
#)